In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
import random
from datetime import datetime
MAX_REQUESTS = 1e4
print("imported")
import pandas as pd

In [ ]:
from queue import Queue
from threading import Thread
import time


def compute(lambda_):
    items = []
    def treat_job(q):
        while True:
            obj = q.get()
            job, s, (t11, t12), (t21,t22) = obj
        
            if job == 1:
                t11 = (datetime.utcnow()-start).total_seconds()*1000
                sleep_time = random.lognormvariate(1.5, 0.6)/1000
                time.sleep(sleep_time)
                t12 = (datetime.utcnow()-start).total_seconds()*1000
                q.put((2, s , (t11, t12), (t21, t22)))

            if job == 2:
                t21 = (datetime.utcnow()-start).total_seconds()*1000
                sleep_time = random.uniform(0.6,1)/1000
                time.sleep(sleep_time)
                t22 = (datetime.utcnow()-start).total_seconds()*1000
                items.append([s , t11, t12, t21, t22])

            q.task_done()

    q = Queue(maxsize=0) 

    worker = Thread(target=treat_job, args=(q,))
    worker.setDaemon(True)
    worker.start()
    start = datetime.utcnow()

    for _ in range(10**4):
        q.put((1, (datetime.utcnow()-start).total_seconds()*1000, (0, 0), (0,0)))
        time.sleep(random.expovariate(lambda_))
    print("Done putting into queue!")
    q.join()
    print("Queue done joining")
    return items

In [ ]:
items = compute(80)

In [ ]:
df = pd.DataFrame(items, columns = ['Start', 'T11', 'T12', 'T21', 'T22'])

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(df['Start'], range(10**4), label="Arrived")
plt.step(df['T22'], range(10**4),label="Served")
plt.title("Number of requests arrived and served")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.step(df['Start'][100:150], range(50), label="Arrived")
plt.step(df['T22'][100:150], range(50), label="Served")
plt.title("Number of requests arrived and served (zoomed)")
plt.legend()
plt.show()

In [ ]:
interesting_times = np.sort(np.concatenate((df['Start'].values, df['T12'].values)))
type1_in_queue = []
type2_in_queue = []
for t in interesting_times:
    type_ones = df[((df['Start'] - t) < 0) & ((df['T12'] - t > 0))]
    type_twos = df[((df['T12'] - t) < 0) & ((df['T22'] - t > 0))]
    type1_in_queue.append(len(type_ones))
    type2_in_queue.append(len(type_twos))

In [ ]:
fig = plt.figure(figsize=(15,5))
fig.add_subplot(2,1,1)
plt.plot(interesting_times, type1_in_queue, c="C0")
plt.title("Type 1 in queue")
fig.add_subplot(2,1,2)
plt.plot(interesting_times, type2_in_queue, c="orange");
plt.title("Type 2 in queue")
plt.tight_layout()

### average response time (event average)

In [ ]:
df['T2-Serv'] = df['T22']-df['T12']
df['T1-Serv'] = df['T12']-df['Start']

In [ ]:
print("Average for type 1: {:.5f} ms".format(df['T1-Serv'].mean()))
print("Average for type 2: {:.5f} ms".format(df['T2-Serv'].mean()))

In [ ]:
plt.bar([1,2],[df['T1-Serv'].mean(), df['T2-Serv'].mean()])

### Average number of jobs served per second

In [ ]:
served = []
for sample_start_time in (df.values.max()-1000)*np.random.sample(50):
    served.append(df[(df['T21'] > sample_start_time) & (df['T22'] < sample_start_time+1000)].count()['Start'])
print(np.mean(served))

In [ ]:
served = []
for sample_start_time in (df.values.max()-1000)*np.random.sample(50):
    served.append(df[(df['T11'] > sample_start_time) & (df['T12'] < sample_start_time+1000)].count()['Start'])
print(np.mean(served))

# 2

In [ ]:
lambdas = np.arange(25, 250, 25)
queues = []

In [ ]:
items = []
#for l in lambdas: 
#    it = compute(l)
#    items.append(it)

In [ ]:
#for i in range(len(items)): 
#    df = pd.DataFrame(items[i], columns = ['Start', 'T11', 'T12', 'T21', 'T22'])
#    df.to_csv("lambda_"+str(25*(i+1)), index=False)

# 3